In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Загружаем датасет

In [4]:
import json

data_file = "/content/drive/MyDrive/TelegramData/telegram_dataset.json"


with open(data_file, 'r') as f:
  data = json.load(f)

len(data), type(data)

(5002, list)

In [5]:
data[999]

{'post_id': 728,
 'post_text': 'Еще хотела напомнить про этот опрос. Мы хотим завтра начать анализировать ответы, поэтому если вы хотели его пройти, пройдите, пожалуйста, в ближайшее время. Спасибо❤️\n\nА обычный контент с разборами статей на этом канале возобновится совсем скоро.',
 'post_date': '2023-05-08 16:13:55+00:00',
 'post_reactions': {'❤': 13, '👍': 8, '✍': 3},
 'comments': [{'comment_id': 6973,
   'text': 'тоже давно прошел ☺️\nзови на тестовый запуск, пожалуйста, если будет такая возможность',
   'date': '2023-05-08 18:53:40+00:00'},
  {'comment_id': 6971,
   'text': 'тоже прошел сразу. Уверен, что получится прекрасный курс!',
   'date': '2023-05-08 17:55:33+00:00'},
  {'comment_id': 6972,
   'text': 'this shit is deeeeead yo',
   'date': '2023-05-08 18:05:56+00:00'}],
 'comment_reactions': [{'❤': 4}, {'❤': 1}, {}]}

In [6]:
corpus = []
for ind, keys in enumerate(data):
    post = keys['post_text']
    comments_list = []
    for _, x in enumerate(keys['comments']):
      comments_list.append(x['text'])

    corpus.append((post, comments_list))

len(corpus), corpus[0]

(5002,
 ('Где заканчивается «Технофеодализм» Варуфакиса // Derrunda\n\nVK l Dzen l Web',
  ['яблочный', 'деррунда это не мы это ты что-то наделал']))

In [7]:
# зависимости
# !pip install torch transformers sentence-transformers scikit-learn

import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split

from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset

In [8]:
# вытягиваем посты и комменты
posts = [x[0] for x in corpus]
# print(posts[:3])

comments = [x[1] for x in corpus]
print(comments[:3])

[['яблочный', 'деррунда это не мы это ты что-то наделал'], ['И веревку', 'И грузовик с ассенизатором', 'Душистое мыло', 'А как вам "Жертвоприношение"? Если он 2013го, то после него у меня было наоборот желание убрать весь лишний хлам и грязь. Как Петрарка который советовал либо раздать все вещи друзьям, либо сжечь.', 'не, не могу смотреть. герман слишком витальный для нашего купороса. сталкера наверно', '❤️'], ['Блядь, чувствую себя андроидом Маркусом, извините', 'Трактовка судеб человеков из прошлого, очень завязана на том, что до середины двадцатого века, большинство людей людьми не считалось, а просвещенные меньшинства в лучшем случае относились к ним как к доморощенному скоту. Уровень страдания должен снижаться, таков путь цивилизации', 'Мне не интересна судьба человечества через тысячу лет. Мне нужно что бы в новой стране которую мы сейчас спасаем своей кровью дети знали свою историю.', 'Совсем недавно наткнулся на паблик и то, в телеге. Но мне нравится читать мысли автора. Пожал 

In [9]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device

device(type='cuda')

In [10]:
model_embs = SentenceTransformer("cointegrated/rubert-tiny2")
model_embs.cuda()


all_comments = [c for com_list in comments
                  for c in com_list]

len(all_comments)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

14431

In [11]:
# кодируем в эмбеддинги комментарии посты
comment_embs = model_embs.encode(all_comments, convert_to_numpy=True)
post_embs = model_embs.encode(posts, convert_to_numpy=True)

# кластеризируем (получим кластеры на основе настроений комментариев.
# можно использовать отдельные модели для сентимент анализа и разбить комментарии прогоном через нее
# либо разбивать на кластеры и отдельно каждый кластер прогонять через сентимент модель.
# после чего для интерпретируемости называть классы по самым популярным настроениям
# так получим более адаптированную группировку по кластерам + интерпретируемость

K = 5
kmeans = KMeans(n_clusters=K, random_state=42)
cluster_ids = kmeans.fit_predict(comment_embs)

cur_idx = 0

ft_dataset = []
for post_idx, pairs in enumerate(corpus):
  post = pairs[0]
  comments_pair = pairs[1]
  for comment in comments_pair:
    ft_dataset.append([post, cluster_ids[cur_idx], comment])
    cur_idx += 1

examples = [
    {
        "input_text": f"Пост: {post} Тип реакции: {cluster}",
        "output_text": comment
    }
    for post, cluster, comment in ft_dataset
]

## 2. Препроцессинг

In [16]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# определяем токен окончания генерации
if tokenizer.eos_token is None:
    tokenizer.eos_token = "</s>"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# препроцесс
def preprocess(ex):
    text = ex["input_text"] + "\nКомментарий: " + ex["output_text"] + tokenizer.eos_token
    tokens = tokenizer(text, truncation=True, padding="max_length", max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# добавление спецтокенов для кластеров
special_tokens = {"additional_special_tokens": [f"<cluster_{i}>" for i in range(K)]}
num_added = tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(50262, 768)

In [17]:
dataset = Dataset.from_list(examples)
dataset = dataset.map(preprocess, batched=False)

# разбиваем выборку
train_test = dataset.train_test_split(test_size=0.2, seed=1)
train_dataset = train_test["train"]
test_dataset = train_test["test"]

# задаем параметры обучения модели
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=False,
    fp16=torch.cuda.is_available(),
    report_to=["tensorboard"],
)

# создаем хф-трейнер
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Map:   0%|          | 0/14431 [00:00<?, ? examples/s]

In [18]:
# запуск обучение
trainer.train()

# сохранение модели
save_dir = "/content/drive/MyDrive/diploma/weights_model"

trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

Epoch,Training Loss,Validation Loss
1,2.847300,2.663432
2,1.949500,2.412147
3,1.701200,2.227776
4,1.572500,2.098223
5,1.454700,2.025734
6,1.318800,1.992783
7,0.992600,1.960294
8,1.033400,1.952969
9,0.895500,1.955486
10,0.816400,1.958703


('/content/drive/MyDrive/diploma/weights_model/tokenizer_config.json',
 '/content/drive/MyDrive/diploma/weights_model/special_tokens_map.json',
 '/content/drive/MyDrive/diploma/weights_model/vocab.json',
 '/content/drive/MyDrive/diploma/weights_model/merges.txt',
 '/content/drive/MyDrive/diploma/weights_model/added_tokens.json',
 '/content/drive/MyDrive/diploma/weights_model/tokenizer.json')

## Проверим результат на примере

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM

load_dir = "/content/drive/MyDrive/diploma/weights_model"

tokenizer = AutoTokenizer.from_pretrained(load_dir)
model = AutoModelForCausalLM.from_pretrained(load_dir).eval()

In [42]:
post_text = """В октябре световой день в Москве сократится на 2 часа 15 минут.

По данным Московского планетария, в начале месяца солнце будет светить 11 часов 34 минуты, а к концу — только 9 часов 19 минут."""
cluster_id = 1  # тип реакции
prompt = f"Пост: {post_text} Тип реакции: <cluster_{cluster_id}>\nКомментарий:"

In [43]:
# если загружать веса модели без обучения, то инпут должен быть на цпу, если сразу после обучения, то нужно перекинуть на cuda
inputs = tokenizer(prompt, return_tensors="pt") #.to(device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(generated_text)

Пост: В октябре световой день в Москве сократится на 2 часа 15 минут.

По данным Московского планетария, в начале месяца солнце будет светить 11 часов 34 минуты, а к концу — только 9 часов 19 минут. Тип реакции: <cluster_1>
Комментарий: Ну так и есть. Только с другой стороны не факт что это произойдёт раньше обычного дня. Если погода позволит то можно загорать) Так что если все пойдут загорать то уже ночью😁 А так да - тоже вариант. Но я бы не стал исключать такую возможность. Тем более сейчас лето чудесное время года! Вечно такая жара</s>


*Note:


модель может выдавать ссылки (вроде случайно сгенерированные), нужно почистить все комментарии и посты от них

